In [1]:
from geopy.geocoders import Nominatim
def get_lat_long_from_address(address):
   locator = Nominatim(user_agent='myGeocoder')
   location = locator.geocode(address)
   return location.latitude, location.longitude
# example
address = '494 Victoria Street South, Kitchener, ON N2M 1Y4, Canada'
get_lat_long_from_address(address)


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=494+Victoria+Street+South%2C+Kitchener%2C+ON+N2M+1Y4%2C+Canada&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [2]:
import requests
import pandas as pd

In [3]:

def get_directions_response(lat1, long1, lat2, long2, mode='bicycle'):
    url = "https://route-and-directions.p.rapidapi.com/v1/routing"
    key = "dea5a77bd2msh6c14e5c2257eed9p19b6b0jsneb1ceff345f2"
    host = "route-and-directions.p.rapidapi.com"
    headers = {
        "X-RapidAPI-Key": key,
        "X-RapidAPI-Host": host
    }
    querystring = {
        "waypoints": f"{lat1},{long1}|{lat2},{long2}",
        "mode": mode
    }
    response = requests.get(url, headers=headers, params=querystring)
    return response

response = get_directions_response(48.34364, 10.87474, 48.37073, 10.90925)
print(response.text)


{"features":[{"type":"Feature","properties":{"mode":"bicycle","waypoints":[{"location":[10.87474,48.34364],"original_index":0},{"location":[10.90925,48.37073],"original_index":1}],"units":"metric","distance":5670,"distance_units":"meters","time":1289.491,"legs":[{"distance":5670,"time":1289.491,"steps":[{"from_index":0,"to_index":43,"distance":633,"time":141.668,"instruction":{"text":"Bike north on Gögginger Straße."}},{"from_index":43,"to_index":130,"distance":1523,"time":352.378,"instruction":{"text":"Bear right to stay on Gögginger Straße."}},{"from_index":130,"to_index":132,"distance":10,"time":6,"instruction":{"text":"Turn right."}},{"from_index":132,"to_index":147,"distance":180,"time":42.767,"instruction":{"text":"Turn left onto Gögginger Straße."}},{"from_index":147,"to_index":170,"distance":270,"time":67.645,"instruction":{"text":"Continue on Hermanstraße."}},{"from_index":170,"to_index":174,"distance":110,"time":22.675,"instruction":{"text":"Turn right onto Völkstraße."}},{"f

In [4]:
# 
address1 = '49 Frederick St, Kitchener, ON N2H 6M7'
address2 = '589 Fairway Rd S, Kitchener, ON N2C 1X4, canada'
addresses = [address1, address2]

In [5]:
lat_lons = [get_lat_long_from_address(addr) for addr in addresses]

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=49+Frederick+St%2C+Kitchener%2C+ON+N2H+6M7&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [ ]:
responses = []
for n in range(len(lat_lons)-1):
   lat1, lon1, lat2, lon2 = lat_lons[n][0], lat_lons[n][1], lat_lons[n+1][0], lat_lons[n+1][1]
   response = get_directions_response(lat1, lon1, lat2, lon2, mode='bicycle')
   responses.append(response)

In [6]:
import folium
def create_map(response):
   # use the response
   mls = response.json()['features'][0]['geometry']['coordinates']
   points = [(i[1], i[0]) for i in mls[0]]
   m = folium.Map()
   # add marker for the start and ending points
   for point in [points[0], points[-1]]:
      folium.Marker(point).add_to(m)
   # add the lines
   folium.PolyLine(points, weight=5, opacity=1).add_to(m)
   # create optimal zoom
   df = pd.DataFrame(mls[0]).rename(columns={0:'Lon', 1:'Lat'})[['Lat', 'Lon']]
   sw = df[['Lat', 'Lon']].min().values.tolist()
   ne = df[['Lat', 'Lon']].max().values.tolist()
   m.fit_bounds([sw, ne])
   return m
m = create_map(response)


In [ ]:
def create_map(responses, lat_lons):
   m = folium.Map()
   df = pd.DataFrame()
   # add markers for the places we visit
   for point in lat_lons:
      folium.Marker(point).add_to(m)
   # loop over the responses and plot the lines of the route
   for response in responses:
      mls = response.json()['features'][0]['geometry']['coordinates']
      points = [(i[1], i[0]) for i in mls[0]]
      
      # add the lines
      folium.PolyLine(points, weight=5, opacity=1).add_to(m)
      temp = pd.DataFrame(mls[0]).rename(columns={0:'Lon', 1:'Lat'})[['Lat', 'Lon']]
      df = pd.concat([df, temp])
   # create optimal zoom
   sw = df[['Lat', 'Lon']].min().values.tolist()
   sw = [sw[0]-0.0005, sw[1]-0.0005]
   ne = df[['Lat', 'Lon']].max().values.tolist()
   ne = [ne[0]+0.0005, ne[1]+0.0005]
   m.fit_bounds([sw, ne])
   return m
m = create_map(responses, lat_lons)
m.save('./route_map.html')
